In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

In [ ]:
from pyspark.ml.feature import *
from pyspark.sql.functions import *

In [ ]:
df1 = spark.read.option("lineSep", ".").text('shared/data/midterm/hw1dir')

In [ ]:
token = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'value', outputCol = 'text')

In [ ]:
df2 = token.transform(df1.dropna())

In [ ]:
hashTF = HashingTF(inputCol = "text", outputCol = "vectors")

In [ ]:
df3 = hashTF.transform(df2)
#df3.show()

In [ ]:
df3 = df3.filter(size((col("text"))) >= 1)

In [ ]:
minHash = MinHashLSH(inputCol="vectors", outputCol="hash")

In [ ]:
model = minHash.fit(df3)

In [ ]:
final1 = model.transform(df3).drop("text")
final2 = model.transform(df3).drop("text")

In [ ]:
model.approxSimilarityJoin(final1, final2, 0.05, distCol="Jaccard").show()